# Getting Started with Ax

Complex optimization problems where we wish to tune multiple parameters to improve metric performance, but the inter-parameter interactions are not fully understood, are common across various fields including machine learning, robotics, materials science, and chemistry.
This category of problem is known as "black-box" optimization.
The complexity of black-box optimization problems further increases if evaluations are expensive to conduct, time-consuming, or noisy.

We can use Ax to efficiently conduct an experiment in which we "ask" for candidate points to evaluate, "tell" Ax the results, and repeat.
We'll uses Ax's `Client`, a tool for managing the state of our experiment, and we'll learn how to define an optimization problem, configure an experiment, run trials, analyze results, and persist the experiment for later use using the `Client`.

Because Ax is a black box optimizer, we can use it to optimize any arbitrary function. In this example we will minimize the [Hartmann6 function](https://www.sfu.ca/~ssurjano/hart6.html), a complicated 6-dimensional function with multiple local minima.
Hartmann6 is a challenging benchmark for optimization algorithms commonly used in the global optimization literature -- it tests the algorithm's ability to identify the true global minimum, rather than mistakenly converging on a local minimum.
Looking at its analytic form we can see that it would be incredibly challenging to efficiently find the global minimum either by manual trial-and-error or traditional design of experiments like grid-search or random-search.

$$
f(\mathbf{x})=-\sum_{i=1}^4 \alpha_i \exp \left(-\sum_{j=1}^6 A_{i j}\left(x_j-P_{i j}\right)^2\right)
$$

### Learning Objectives
- Understand the basic concepts of black box optimization
- Learn how to define an optimization problem using Ax
- Configure and run an experiment using Ax's `Client`
- Analyze the results of the optimization

### Prerequisites

* Familiarity with Python and basic programming concepts
* Understanding of [adaptive experimentation](../../intro-to-ae.mdx) and [Bayesian optimization](../../intro-to-bo.mdx)


## Step 1: Import Necessary Modules

First, ensure you have all the necessary imports:

In [ ]:
import numpy as np
from ax.api.client import Client
from ax.api.configs import RangeParameterConfig

## Step 2: Initialize the Client

Create an instance of the `Client` to manage the state of your experiment.

In [ ]:
client = Client()

## Step 3: Configure the Experiment

The `Client` instance can be configured with a series of `Config`s that define how the experiment will be run.

The Hartmann6 problem is usually evaluated on the hypercube $x_i \in (0, 1)$, so we will define six identical `RangeParameterConfig`s with these bounds.

You may specify additional features like parameter constraints to further refine the search space and parameter scaling to help navigate parameters with nonuniform effects.


In [ ]:
# Define six float parameters x1, x2, x3, ... for the Hartmann6 function, which is typically evaluated on the unit hypercube
parameters = [
    RangeParameterConfig(
        name="x1", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x2", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x3", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x4", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x5", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x6", parameter_type="float", bounds=(0, 1)
    ),
]

client.configure_experiment(parameters=parameters)

## Step 4: Configure Optimization
Now, we must configure the objective for this optimization, which we do using `Client.configure_optimization`.
This method expects a string `objective`, an expression containing either a single metric to maximize, a linear combination of metrics to maximize, or a tuple of multiple metrics to jointly maximize.
These expressions are parsed using [SymPy](https://www.sympy.org/en/index.html). For example:
* `"score"` would direct Ax to maximize a metric named score
* `"-loss"` would direct Ax to Ax to minimize a metric named loss
* `"task_0 + 0.5 * task_1"` would direct Ax to maximize the sum of two task scores, downweighting task_1 by a factor of 0.5
* `"score, -flops"` would direct Ax to simultaneously maximize score while minimizing flops

See these recipes for more information on configuring [objectives](../../recipes/multi-objective-optimization) and [outcome constraints](../../recipes/outcome-constraints).

In [ ]:
metric_name = "hartmann6" # this name is used during the optimization loop in Step 5
objective = f"-{metric_name}" # minimization is specified by the negative sign

client.configure_optimization(objective=objective)

## Step 5: Run Trials
Here, we will configure the ask-tell loop.

We begin by defining the Hartmann6 function as written above.
Remember, this is just an example problem and any Python function can be substituted here.

In [ ]:
# Hartmann6 function
def hartmann6(x1, x2, x3, x4, x5, x6):
    alpha = np.array([1.0, 1.2, 3.0, 3.2])
    A = np.array([
        [10, 3, 17, 3.5, 1.7, 8],
        [0.05, 10, 17, 0.1, 8, 14],
        [3, 3.5, 1.7, 10, 17, 8],
        [17, 8, 0.05, 10, 0.1, 14]
    ])
    P = 10**-4 * np.array([
        [1312, 1696, 5569, 124, 8283, 5886],
        [2329, 4135, 8307, 3736, 1004, 9991],
        [2348, 1451, 3522, 2883, 3047, 6650],
        [4047, 8828, 8732, 5743, 1091, 381]
    ])

    outer = 0.0
    for i in range(4):
        inner = 0.0
        for j, x in enumerate([x1, x2, x3, x4, x5, x6]):
            inner += A[i, j] * (x - P[i, j])**2
        outer += alpha[i] * np.exp(-inner)
    return -outer

hartmann6(0.1, 0.45, 0.8, 0.25, 0.552, 1.0)

### Optimization Loop

We will iteratively call `client.get_next_trials` to "ask" Ax for a parameterization to evaluate, then call `hartmann6` using those parameters, and finally "tell" Ax the result using `client.complete_trial`.

This loop will run multiple trials to optimize the function.

In [ ]:
for _ in range(10): # Run 10 rounds of trials
    # We will request three trials at a time in this example
    trials = client.get_next_trials(max_trials=3)

    for trial_index, parameters in trials.items():
        x1 = parameters["x1"]
        x2 = parameters["x2"]
        x3 = parameters["x3"]
        x4 = parameters["x4"]
        x5 = parameters["x5"]
        x6 = parameters["x6"]

        result = hartmann6(x1, x2, x3, x4, x5, x6)

        # Set raw_data as a dictionary with metric names as keys and results as values
        raw_data = {metric_name: result}

        # Complete the trial with the result
        client.complete_trial(trial_index=trial_index, raw_data=raw_data)

## Step 6: Analyze Results

After running trials, you can analyze the results.
Most commonly this means extracting the parameterization from the best performing trial you conducted.

Hartmann6 has a known global minimum of $f(x*) = -3.322$ at $x* = (0.201, 0.150, 0.477, 0.273, 0.312, 0.657)$.
Ax is able to identify a point very near to this true optimum **using just 30 evaluations.**
This is possible due to the sample-efficiency of [Bayesian optimization](../../intro-to-bo), the optimization method we use under the hood in Ax.

In [ ]:
best_parameters, prediction, index, name = client.get_best_parameterization()
print("Best Parameters:", best_parameters)
print("Prediction (mean, variance):", prediction)

## Step 7: Compute Analyses

Ax can also produce a number of analyses to help interpret the results of the experiment via `client.compute_analyses`.
Users can manually select which analyses to run, or can allow Ax to select which would be most relevant.
In this case Ax selects the following:
* **Arm Effects Plots** show the metric value for each [arm](https://ax.dev/docs/next/glossary#arm) on the experiment. Ax produces one plot using values from its internal surrogate model (this can be helpful for seeing the true effect of an arm when evaluations are noisy) and another using the raw metric values as observed.
* **Summary** lists all trials generated along with their parameterizations, observations, and miscellaneous metadata
* **Sensitivity Analysis Plot** shows which parameters have the largest affect on the objective using [Sobol Indicies](https://en.wikipedia.org/wiki/Variance-based_sensitivity_analysis)
* **Slice Plot** shows how the model predicts a single parameter effects the objective along with a confidence interval
* **Contour Plot** shows how the model predicts a pair of parameters effects the objective as a 2D surface
* **Cross Validation** helps to visualize how well the surrogate model is able to predict out of sample points 

In [ ]:
# display=True instructs Ax to sort then render the resulting analyses
cards = client.compute_analyses(display=True)

## Conclusion

This tutorial demonstrates how to use Ax's `Client` for ask-tell optimization of Python functions using the Hartmann6 function as an example.
You can adjust the function and parameters to suit your specific optimization problem.